In [2]:
import pandas as pd
from sqlalchemy import create_engine
import pymysql
import os

# 創資料庫

In [ ]:
# def createSQLDatabase(newDatabaseName):
#     db = pymysql.connect(host='localhost', port=3306, user='root', passwd='benbilly3@', charset='utf8')
#     cursorInsatnce  =db.cursor()  
#     # SQL Statement to create a database
#     sqlStatement = "CREATE DATABASE "+newDatabaseName  
#     # Execute the create database SQL statment through the cursor instance
#     cursorInsatnce.execute(sqlStatement)
# createSQLDatabase('settleProcess')

# 資料庫連線

In [9]:
connect_info = 'mysql+pymysql://{}:{}@{}:{}/{}?charset=utf8'.format('root','password','localhost',3306,'settlement_test')  #1
engine = create_engine(connect_info)

# 匯入測試資料

In [ ]:
df=pd.read_csv('/Users/benbilly3/Desktop/Innopro-master/Innopro_crawl_data/201908/留言大表測試範例.csv')
df.to_sql(name='orginalBigSheet', con=engine, if_exists='append')

In [ ]:
df2=pd.read_csv('/Users/benbilly3/Desktop/Innopro-master/Innopro_crawl_data/201908/T團-8月團購規則.xlsx - settlementRule.csv')
df2.to_sql(name='settlementRule', con=engine, if_exists='append')

In [ ]:
df3=pd.read_csv('/Users/benbilly3/Desktop/Innopro-master/Innopro_crawl_data/201908/T團-8月團購規則.xlsx - deliverRule.csv')
df3.to_sql(name='deliverRule', con=engine, if_exists='append')

In [ ]:
df4=pd.read_csv('/Users/benbilly3/Desktop/Innopro-master/Innopro_crawl_data/201908/T團-8月團購規則.xlsx - deliverFreeAllRule.csv')
df4.to_sql(name='deilverFreeAllRule', con=engine, if_exists='append')

# 結算測試

## 處理限定+1商品,訂購數大於限制數且限制數大於0
## 計算買Ｘ送Ｙ後的訂購數

### 重要欄位說明

orginalOrderCount=原請求數

orderCount=限加處理後瓶數

giftBottle=贈送瓶數

orderCountAfterSale=贈送後總瓶數

In [23]:
def limitAndGiftBottleProcess(session,method='append'):
    
#若為第一次建資料，拿掉判斷是式
#     check_exist=pd.read_sql('select * from limitAndGiftBottleProcess where session='+str(session),con=engine,index_col=['wineID'])#防止重複結算
#     check_exist['session']=check_exist['session'].astype(str)
#     if session not in check_exist['session'].values:
    df=pd.read_sql('select session,wineID,comment_order,username,fbuid,orderCount from orginalBigSheet where session='+str(session),con=engine,index_col=['wineID'])
    df2=pd.read_sql('select * from settlementRule where session='+str(session),con=engine,index_col=['wineID'])
    for i in df2.columns[3:]:
        df[i]=df2[i]
    df.index=df.index.astype(int).astype(str)#索引轉文字格式
    df['orginalOrderCount']=df['orderCount']
    #處理限定+1商品,訂購數大於限制數且限制數大於0
    df['orderCount']=[b if (b>0)&(a>b) else a for a,b in zip(df['orderCount'],df['limit'])]
    #計算買Ｘ送Ｙ後的訂購數
    df['saleBasic']=df['sale'].apply(lambda s:pd.to_numeric(s[:-2]) if str(s)!='0' else 1)#買Ｘ,else=1，讓無優惠品項製造基底為1
    df['saleNum']=df['sale'].apply(lambda s:pd.to_numeric(s[-1]) if str(s)!='0' else 0)#送Ｙ,else=，，讓無優惠品項歸0
    #計算贈送瓶數
    df['giftBottle']=df['orderCount']//(df['saleBasic']+df['saleNum'])
    df['giftBottle']=[0 if a==b else a for a,b in zip(df['giftBottle'],df['orderCount'])]
    #計算計價基準數量
    df['orderCountAfterSale']=df['orderCount']-df['giftBottle']#原瓶數除優惠基底*加送變數+原瓶數=優惠後訂購數
    df=df.reset_index()#解索引才能存SQL
    df.to_sql(name='limitAndGiftBottleProcess', con=engine, if_exists=method)#if_exists='append 可疊加不同資料
#     else:
#         print('limitAndGiftBottleProcess session is duplicated')
    return df
a=limitAndGiftBottleProcess('201908')
a


,wineID,session,comment_order,username,fbuid,orderCount,price,stockNum,wineName,style,deliverFree,sale,limit,orginalOrderCount,saleBasic,saleNum,giftBottle,orderCountAfterSale
0,3053,201908,5,Alex Kao,573504806,1,1680,44,Caiarossa 2010 IGT Tuscany 紅酒,0,0,0,0,1,1,0,0,1
1,3053,201908,1,Chialin Liao,1133912470,1,1680,44,Caiarossa 2010 IGT Tuscany 紅酒,0,0,0,0,1,1,0,0,1
2,3053,201908,6,Don Wu,100001491233237,1,1680,44,Caiarossa 2010 IGT Tuscany 紅酒,0,0,0,0,1,1,0,0,1
3,3053,201908,3,Jessica LuLu,592053470,1,1680,44,Caiarossa 2010 IGT Tuscany 紅酒,0,0,0,0,1,1,0,0,1
4,3053,201908,4,鄭建中,100007536616399,1,1680,44,Caiarossa 2010 IGT Tuscany 紅酒,0,0,0,0,1,1,0,0,1
5,3053,201908,0,Shen Lin,688753070,1,1680,44,Caiarossa 2010 IGT Tuscany 紅酒,0,0,0,0,1,1,0,0,1
6,3053,201908,7,Simon Chuang,1331272331,1,1680,44,Caiarossa 2010 IGT Tuscany 紅酒,0,0,0,0,1,1,0,0,1
7,3053,201908,2,Yi-Hung Pan,100000332146403,1,1680,44,Caiarossa 2010 IGT Tuscany 紅酒,0,0,0,0,1,1,0,0,1
8,4344,201908,17,Albert Lai,100000399041054,1,490,120,Stephane Aviron 2016 “Moulin a Vent” Vieilles ...,0,0,0,0,1,1,0,0,1
9,4344,201908,14,Alex Kao,573504806,2,490,120,Stephane Aviron 2016 “Moulin a Vent” Vieilles ...,0,0,0,0,2,1,0,0,2


# confirmList

### 決定購買權，確認購買清單

orderCountCumsum=訂購累加數

getSure=顯示有無買到(sorry為沒買到,get為買到)

totalPrice=個人單項商品總價

orderCount_sum=單項商品總訂購數

In [27]:
def confirmPurchaseRight(session,method='append'):
#     check_exist=pd.read_sql('select * from confirmPurchaseRight where session='+str(session),con=engine,index_col=['wineID'])#防止重複結算
#     check_exist['session']=check_exist['session'].astype(str)
#     if session not in check_exist['session'].values:
    df=pd.read_sql('select * from limitAndGiftBottleProcess where session='+str(session),con=engine,index_col=['wineID'])
    d=[]
    for wineID in list(dict.fromkeys(df.index)):
        try:
            if df.index.tolist().count(wineID)<2:#只有一篇留言的單獨抓出,因為是series要獨自處理
                t=df.iloc[df.index.tolist().index(wineID):df.index.tolist().index(wineID)+1]#Series轉dataframe
            else:
                t=df.loc[wineID]
                t=t.sort_values(["comment_order"])#留言排序
            t['orderCountCumsum']=t['orderCount'].cumsum()#累加數量
            cond1=t['orderCountCumsum']<=t['stockNum']#設定安全名單條件
            t_in=t[cond1]#安全名單
            t_out=t[~cond1]#排除名單
            #最後1人order的數量大於剩餘庫存(t_out名單第一人)，則電腦自動更新其最大可買數
            if ((t_in['stockNum']-t_in['orderCountCumsum']).iloc[-1]>0) and len(t_out)>0:#安全名單未滿足庫存,且還有人在排隊時
                t_out['orderCount'].iloc[0]=((t_in['stockNum']-t_in['orderCountCumsum']).iloc[-1])#調整最後一人數量
                final=pd.concat([t_in,t_out.iloc[:1]])#把最後一人再加回
                t_out=t_out.iloc[1:]
                final=pd.concat([final,t_out])#名單重整合併
            else:
                final=pd.concat([t_in,t_out])#名單重整合併
        except:
            print('error',wineID)#錯誤回報
            pass
        final['orderCountCumsum']=final['orderCount'].cumsum()#重新加總購買量
        s=final['orderCountCumsum']-final['stockNum']#顯示有無無買到
        final['getSure']=['get' if i<=0 else 'sorry' for i in s]#顯示有無無買到

        d.append(final)
    result=pd.concat(d)
    
    #最後一位可買人若購買量小於庫存量，購買數會自動變成最大可買量，會影響買X送Y優惠條件，必須再算一次
    #計算買Ｘ送Ｙ後的訂購數
    #計算贈送瓶數
    result['giftBottle']=result['orderCount']//(result['saleBasic']+result['saleNum'])
    result['giftBottle']=[0 if a==b else a for a,b in zip(result['giftBottle'],result['orderCount'])]
    #計算計價基準數量
    result['orderCountAfterSale']=result['orderCount']-result['giftBottle']#原瓶數除優惠基底*加送變數+原瓶數=優惠後訂購數
    #sorry 的數量調成0
    result['orderCount']=[0 if a=='sorry'else b for a,b in zip(result['getSure'],result['orderCount'])]
    result['orderCountAfterSale']=[0 if a=='sorry'else b for a,b in zip(result['getSure'],result['orderCountAfterSale'])]
    
    result['totalPrice']=result['orderCountAfterSale']*result['price']
    
    
    result['orderCount_sum']=result.groupby(['wineID'])['orderCount'].sum()#商品銷售總量
    result= result[result['orginalOrderCount']!=0]#刪掉留言為0的
    result=result.reset_index()
    if 'index' in result.columns.values:
        result=result.drop(columns=['index'])
    result.to_sql(name='confirmPurchaseRight', con=engine, if_exists=method)
        
#     else:
#         print('confirmPurchaseRight session is duplicated')
    return result


confirmPurchaseRight('201908')

/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: Se

,wineID,session,comment_order,username,fbuid,orderCount,price,stockNum,wineName,style,...,limit,orginalOrderCount,saleBasic,saleNum,giftBottle,orderCountAfterSale,orderCountCumsum,getSure,totalPrice,orderCount_sum
0,3053,201908,0,Shen Lin,688753070,1,1680,44,Caiarossa 2010 IGT Tuscany 紅酒,0,...,0,1,1,0,0,1,1,get,1680,8
1,3053,201908,1,Chialin Liao,1133912470,1,1680,44,Caiarossa 2010 IGT Tuscany 紅酒,0,...,0,1,1,0,0,1,2,get,1680,8
2,3053,201908,2,Yi-Hung Pan,100000332146403,1,1680,44,Caiarossa 2010 IGT Tuscany 紅酒,0,...,0,1,1,0,0,1,3,get,1680,8
3,3053,201908,3,Jessica LuLu,592053470,1,1680,44,Caiarossa 2010 IGT Tuscany 紅酒,0,...,0,1,1,0,0,1,4,get,1680,8
4,3053,201908,4,鄭建中,100007536616399,1,1680,44,Caiarossa 2010 IGT Tuscany 紅酒,0,...,0,1,1,0,0,1,5,get,1680,8
5,3053,201908,5,Alex Kao,573504806,1,1680,44,Caiarossa 2010 IGT Tuscany 紅酒,0,...,0,1,1,0,0,1,6,get,1680,8
6,3053,201908,6,Don Wu,100001491233237,1,1680,44,Caiarossa 2010 IGT Tuscany 紅酒,0,...,0,1,1,0,0,1,7,get,1680,8
7,3053,201908,7,Simon Chuang,1331272331,1,1680,44,Caiarossa 2010 IGT Tuscany 紅酒,0,...,0,1,1,0,0,1,8,get,1680,8
8,4344,201908,0,潘宥瑞,100002501697401,1,490,120,Stephane Aviron 2016 “Moulin a Vent” Vieilles ...,0,...,0,1,1,0,0,1,1,get,490,33
9,4344,201908,1,Chung Yu Chen,100000128365914,1,490,120,Stephane Aviron 2016 “Moulin a Vent” Vieilles ...,0,...,0,1,1,0,0,1,2,get,490,33


# 運費規則計算函式

In [28]:
def calDiliverRate(num,n1,p1,n2,p2,n3,p3):
    if n1==p1==n2==p2==n3==p3==0:#若全為0則該類別全免運
        d_p=num*0
    else:
        if num==n1:
            d_p=p1
        else:
            if n1<num<=n2:
                d_p=p2
            elif n2<num<=n3:
                d_p=p3
            else:
                q=num//n3
                r=num%n3
                if 0<r<=n2:
                    d_p=p2+q*p3
                elif r==0:
                    d_p=q*p3
                else:
                    d_p=(q+1)*p3
    return d_p

calDiliverRate(0,1,160,6,225,12,290)

0

# 第一階段運費計算

各類商品分開算運費

In [29]:
def calDiliverCost(session,method='append'):#
#     check_exist=pd.read_sql('select * from calDiliverCost where session='+str(session),con=engine)#防止重複結算
#     check_exist['session']=check_exist['session'].astype(str)
#     if session not in check_exist['session'].values:
    
    df=pd.read_sql('select * from confirmPurchaseRight where session='+str(session),con=engine,index_col=['wineID'])
    df['session']=df['session'].astype(str)
    
    #只納入有購買權的
    df=df[df['getSure']=='get']
    #納入單品項面運費優惠計算(X瓶以上免運)
    df['orderCountAfterDiliverFree']= [0 if (a>0)&(b>=a) else b for a,b in zip(df['deliverFree'],df['orderCount'])]


    #依照個人ID、團次、商品屬性計算運費和總價
    df=df.groupby(['fbuid','session','style'])['orderCount','orderCountAfterDiliverFree','totalPrice'].sum()
    df=df.reset_index()
    df['session']=df['session'].astype(str)
    df['style']=df['style'].astype(str)
    df=df.set_index(['session','style'])
    
    #引入運費規則表
    deilverRule=pd.read_sql('select * from deliverRule where session='+str(session),con=engine)
    deilverRule['session']=deilverRule['session'].astype(str)
    deilverRule['style']=deilverRule['style'].astype(str)
    deilverRule=deilverRule.set_index(['session','style'])
    
    for i in deilverRule.columns[-6:]:
        df[i]=deilverRule[i]
        
    #計算各類商品運費
    df['diliverCost']=[calDiliverRate(i,d1,p1,d2,p2,d3,p3)
                           for i,d1,p1,d2,p2,d3,p3 in zip(
                                        df['orderCountAfterDiliverFree'],
                                        df['deliverRule1'],df['deliverRule1Price'],
                                        df['deliverRule2'],df['deliverRule2Price'],
                                        df['deliverRule3'],df['deliverRule3Price']                           
                                                                 )]
    #自取總價
    df['selfGetCost']=df['totalPrice']
    df=df.reset_index()#解索引才能存SQL
    df.to_sql(name='calDiliverCost', con=engine, if_exists=method)#if_exists='append 可疊加不同資料
#     else:
#         print('calDiliverCost session is duplicated')
    return df

calDiliverCost('201908')

,session,style,fbuid,orderCount,orderCountAfterDiliverFree,totalPrice,deliverRule1,deliverRule1Price,deliverRule2,deliverRule2Price,deliverRule3,deliverRule3Price,diliverCost,selfGetCost
0,201908,0,36914827,4,4,4700,1,160,6,225,12,290,225,4700
1,201908,0,501609342,8,8,6580,1,160,6,225,12,290,290,6580
2,201908,0,507948606,2,2,13000,1,160,6,225,12,290,225,13000
3,201908,0,513030038,3,3,1880,1,160,6,225,12,290,225,1880
4,201908,0,518125422,4,4,3260,1,160,6,225,12,290,225,3260
5,201908,0,519998038,1,1,300,1,160,6,225,12,290,160,300
6,201908,0,524402757,3,3,3660,1,160,6,225,12,290,225,3660
7,201908,0,525152873,2,2,1290,1,160,6,225,12,290,225,1290
8,201908,0,526487238,1,1,7710,1,160,6,225,12,290,160,7710
9,201908,0,526497993,1,1,790,1,160,6,225,12,290,160,790


# 全館優惠最終結算

## 各類商品運費合併

selfGetCost=折扣前總價

diliverCostAfterDiscountAll=全館折扣後總價(自取價)

diliverCostAfterDeilverFreeAll=最終運費

totalPriceIncludeDC=最終含運總價

In [30]:
def calDiliverCostFinal(session,method='append'):
#     check_exist=pd.read_sql('select * from calDiliverCostFinal where session='+str(session),con=engine)#防止重複結算
#     check_exist['session']=check_exist['session'].astype(str)
#     if session not in check_exist['session'].values:
    
    df=pd.read_sql('select * from calDiliverCost where session='+str(session),con=engine)
    df['session']=df['session'].astype(str)
    df=df.set_index(['session','style'])
    
    df=df.groupby(['session','fbuid'])['orderCount','selfGetCost','diliverCost'].sum()
    df=df.reset_index()
    df=df.set_index(['session'])
    #引入運費規則表
    Rule=pd.read_sql('select * from deilverFreeAllRule where session='+str(session),con=engine)
    Rule['session']=Rule['session'].astype(str)
    Rule=Rule.set_index('session') 
    for i in Rule.columns[:]:
        df[i]=Rule[i]
        
    #全館折扣
    df['diliverCostAfterDiscountAll']=df['selfGetCost']*df['discountAll']
    
    #全館免運優惠，若買進清單同時有單品運費優惠，則先以全館免運為優先，不適用的話，再用原單品優惠
    df['diliverCostAfterDeilverFreeAll']=[0 if ((a>=e)&(e>0))or((b>=d)&(d>0)) else c
                                          for a,b,c,d,e in zip(df['orderCount'],df['diliverCostAfterDiscountAll'],
                                          df['diliverCost'],df['deilverFreeAll'],df['deilverFreeAllbottle'])]
                                 
#     千辛萬苦弄出的總價
    df['totalPriceIncludeDC']=df['diliverCostAfterDiscountAll']+df['diliverCostAfterDeilverFreeAll']
    df=df.reset_index()#解索引才能存SQL
    if 'index' in df.columns.values:
        df=df.drop(columns=['index'])
    df.to_sql(name='calDiliverCostFinal', con=engine, if_exists=method)#if_exists='append 可疊加不同資料
    
#     else:
#         print('calDiliverCostFinal session is duplicated')
    return df

calDiliverCostFinal('201908')

,session,fbuid,orderCount,selfGetCost,diliverCost,deilverFreeAll,deilverFreeAllbottle,discountAll,diliverCostAfterDiscountAll,diliverCostAfterDeilverFreeAll,totalPriceIncludeDC
0,201908,36914827,4,4700,225,0,6,1,4700,225,4925
1,201908,501609342,8,6580,290,0,6,1,6580,0,6580
2,201908,507948606,2,13000,225,0,6,1,13000,225,13225
3,201908,513030038,3,1880,225,0,6,1,1880,225,2105
4,201908,518125422,4,3260,225,0,6,1,3260,225,3485
5,201908,519998038,1,300,160,0,6,1,300,160,460
6,201908,524402757,3,3660,225,0,6,1,3660,225,3885
7,201908,525152873,2,1290,225,0,6,1,1290,225,1515
8,201908,526487238,1,7710,160,0,6,1,7710,160,7870
9,201908,526497993,1,790,160,0,6,1,790,160,950


# 資料檢視

In [ ]:
# session='201908'
# df=pd.read_sql('select * from confirmPurchaseRight where session='+str(session),con=engine,index_col=['wineID'])
# s=df[df['getSure']=='sorry']
# s